<a href="https://colab.research.google.com/github/nipunnirmal21/Image-Classifier-CNN-/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy -q
!pip install pandas -q
!pip install matplotlib -q
!pip install tensorflow -q

!pip install opendatasets -q

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

import opendatasets as od


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dineshpiyasamara/cats-and-dogs-for-classification")

print("Path to dataset files:", path)

100%|██████████| 217M/217M [00:01<00:00, 137MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/dineshpiyasamara/cats-and-dogs-for-classification/versions/2


In [5]:
BATCH_SIZE = 32
IMAGE_SIZE = (128,128)

In [7]:
train_data_dir = "/root/.cache/kagglehub/datasets/dineshpiyasamara/cats-and-dogs-for-classification/versions/2/cats_dogs/train"
test_data_dir = "/root/.cache/kagglehub/datasets/dineshpiyasamara/cats-and-dogs-for-classification/versions/2/cats_dogs/test"

train_data = tf.keras.utils.image_dataset_from_directory(train_data_dir,
                                                         batch_size=BATCH_SIZE,
                                                         image_size=IMAGE_SIZE,
                                                         subset='training',
                                                         validation_split=0.1,
                                                         seed=42)

validation_data = tf.keras.utils.image_dataset_from_directory(train_data_dir,
                                                         batch_size=BATCH_SIZE,
                                                         image_size=IMAGE_SIZE,
                                                         subset='validation',
                                                         validation_split=0.1,
                                                         seed=42)

test_data = tf.keras.utils.image_dataset_from_directory(test_data_dir,
                                                         batch_size=BATCH_SIZE,
                                                         image_size=IMAGE_SIZE)

Found 8000 files belonging to 2 classes.
Using 7200 files for training.
Found 8000 files belonging to 2 classes.
Using 800 files for validation.
Found 2000 files belonging to 2 classes.


In [8]:
class_names = train_data.class_names
class_names

['cats', 'dogs']

In [9]:
for image_batch,label_batch in train_data.take(1):
    print(image_batch.shape)
    print(label_batch.shape)

(32, 128, 128, 3)
(32,)
